# Embedded Line Chart

Example code generated using help from here: https://python-pptx.readthedocs.io/en/latest/user/charts.html

In [15]:
from pptx import Presentation
from pptx.chart.data import ChartData
from pptx.enum.chart import XL_CHART_TYPE
from pptx.enum.chart import XL_TICK_MARK
from pptx.enum.dml import MSO_LINE
from pptx.enum.chart import XL_LEGEND_POSITION
from pptx.util import Inches
from pptx.dml.color import RGBColor
from pptx.util import Pt

import pandas as pd

# open the presentation
presentation = Presentation()

In [16]:
# create a slide
for i, sl in enumerate(presentation.slide_layouts):
    print(i, sl.name)
SLD_LAYOUT_TITLE_AND_CONTENT = 1
SLD_LAYOUT_TITLE_ONLY = 5
slide_layout = presentation.slide_layouts[SLD_LAYOUT_TITLE_ONLY]
slide = presentation.slides.add_slide(slide_layout)

0 Title Slide
1 Title and Content
2 Section Header
3 Two Content
4 Comparison
5 Title Only
6 Blank
7 Content with Caption
8 Picture with Caption
9 Title and Vertical Text
10 Vertical Title and Text


In [17]:
# list placeholders
for shape in slide.placeholders:
    print('%d %s' % (shape.placeholder_format.idx, shape.name))

0 Title 1


In [18]:
# set the title
title_placeholder = slide.shapes.title
title_placeholder.text = 'Line Chart'

In [19]:
# create the mock-up data
df = pd.DataFrame({
    'quarter': ['Q1', 'Q2', 'Q3', 'Q4'],
    'marketing_accounts': [14, 26, 82, 54],
    'baseline_accounts': [20, 22, 24, 56],
})

colors = {}
colors['Marketing Accounts'] = '3C2F80'
colors['Baseline Accounts'] = '140948'
line_type = {}
line_type['Marketing Accounts'] = MSO_LINE.SOLID
line_type['Baseline Accounts'] = MSO_LINE.LONG_DASH
line_width = {}
line_width['Marketing Accounts'] = 2 * 20116800/1584
line_width['Baseline Accounts'] = 1 * 20116800/1584

df

,quarter,marketing_accounts,baseline_accounts
0,Q1,14,20
1,Q2,26,22
2,Q3,82,24
3,Q4,54,56


In [20]:
# add the data to the chart element
chart_data = ChartData()
chart_data.categories = df['quarter']
chart_data.add_series('Marketing Accounts', df['marketing_accounts'])
chart_data.add_series('Baseline Accounts', df['baseline_accounts'])

In [21]:
# create the chart
x, y, cx, cy = Inches(2), Inches(2), Inches(6), Inches(4.5)
chart = slide.shapes.add_chart(
    XL_CHART_TYPE.LINE, x, y, cx, cy, chart_data
).chart

In [22]:
# # set gap width
# chart.plots[0].gap_width = 100

In [23]:
# set the chart title
chart.has_title = True
chart.chart_title.has_text_frame = True
chart.chart_title.text_frame.text = 'Accounts by Quarter'
chart.chart_title.text_frame.paragraphs[0].font.name = 'Arial'
chart.chart_title.text_frame.paragraphs[0].font.size = Pt(11)

In [24]:
# update the axes
category_axis = chart.category_axis
category_axis.has_major_gridlines = False
category_axis.major_tick_mark = XL_TICK_MARK.NONE
category_axis.minor_tick_mark = XL_TICK_MARK.NONE
category_axis.tick_labels.font.size = Pt(9)
category_axis.tick_labels.font.name = 'Arial'
category_axis.format.line.fill.background()

value_axis = chart.value_axis
#value_axis.maximum_scale = 50.0
value_axis.major_tick_mark = XL_TICK_MARK.NONE
value_axis.minor_tick_mark = XL_TICK_MARK.NONE
value_axis.has_major_gridlines = True
value_axis.has_minor_gridlines = False
value_axis.visible = True


chart.value_axis.major_gridlines.format.line.dash_style = MSO_LINE.LONG_DASH
chart.value_axis.major_gridlines.format.line.width = 0
chart.value_axis.major_gridlines.format.line.color.rgb = RGBColor.from_string('BFBFBF')

tick_labels = value_axis.tick_labels
#tick_labels.number_format = '0"%"'
#tick_labels.font.bold = True
#tick_labels.font.size = Pt(14)

In [25]:
# add data labels
# plot = chart.plots[0]
# plot.has_data_labels = True
# data_labels = plot.data_labels
# data_labels.font.name = 'Arial'
# data_labels.font.size = Pt(11)
# data_labels.font.color.rgb = RGBColor.from_string(colors['Marketing Accounts'])

In [26]:
# adjust the legend
chart.has_legend = True
legend = chart.legend
legend.font.name = 'Arial'
legend.font.size = Pt(9)
legend.horz_offset = 0.2
legend.position = XL_LEGEND_POSITION.BOTTOM

In [27]:
for series in chart.series:
    if series.name in colors:
        line = series.format.line
        line.color.rgb = RGBColor.from_string(colors[series.name])
        line.dash_style = line_type[series.name]
        line.width = int(line_width[series.name])

In [28]:
# save and close
presentation.save('line_chart.pptx')